In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip "/content/drive/MyDrive/Mini project/archive.zip" -d "/content/ravdess"

Archive:  /content/drive/MyDrive/Mini project/archive.zip
  inflating: /content/ravdess/Actor_01/03-01-01-01-01-01-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-01-01-01-02-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-01-01-02-01-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-01-01-02-02-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-02-01-01-01-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-02-01-01-02-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-02-01-02-01-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-02-01-02-02-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-02-02-01-01-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-02-02-01-02-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-02-02-02-01-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-02-02-02-02-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-03-01-01-01-01.wav  
  inflating: /content/ravdess/Actor_01/03-01-03-01-01-02-01.wav  
  inflating: /cont

In [4]:
import os
print(os.listdir("/content/ravdess"))

['Actor_05', 'Actor_03', 'Actor_07', 'Actor_20', 'Actor_22', 'Actor_04', 'Actor_15', 'Actor_02', 'Actor_11', 'Actor_08', 'Actor_23', 'Actor_21', 'Actor_09', 'Actor_19', 'audio_speech_actors_01-24', 'Actor_01', 'Actor_17', 'Actor_06', 'Actor_12', 'Actor_24', 'Actor_16', 'Actor_10', 'Actor_18', 'Actor_13', 'Actor_14']


In [5]:
!pip install librosa tensorflow scikit-learn pandas numpy soundfile tqdm


In [6]:
import librosa
import numpy as np
import pandas as pd
import glob
import os
from tqdm import tqdm



In [7]:
DATA_DIR = "/content/ravdess"
print("Files inside ravdess folder:", os.listdir(DATA_DIR))


Files inside ravdess folder: ['Actor_05', 'Actor_03', 'Actor_07', 'Actor_20', 'Actor_22', 'Actor_04', 'Actor_15', 'Actor_02', 'Actor_11', 'Actor_08', 'Actor_23', 'Actor_21', 'Actor_09', 'Actor_19', 'audio_speech_actors_01-24', 'Actor_01', 'Actor_17', 'Actor_06', 'Actor_12', 'Actor_24', 'Actor_16', 'Actor_10', 'Actor_18', 'Actor_13', 'Actor_14']


In [8]:
files = glob.glob(DATA_DIR + '/**/*.wav', recursive=True)
print("Number of .wav files found:", len(files))
print("Example file:", files[0])


Number of .wav files found: 2880
Example file: /content/ravdess/Actor_05/03-01-02-02-01-01-05.wav


In [9]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=3, offset=0.5)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfccs = np.mean(mfcc.T, axis=0)
    return mfccs


In [10]:
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

X, y = [], []

for file in tqdm(files):
    basename = os.path.basename(file)
    emotion_code = basename.split("-")[2]
    emotion = emotion_map.get(emotion_code, "unknown")

    # Distress emotions → label 1, others → 0
    distress = 1 if emotion in ['sad', 'angry', 'fearful', 'disgust'] else 0

    features = extract_features(file)
    X.append(features)
    y.append(distress)

X = np.array(X)
y = np.array(y)

print("✅ Features shape:", X.shape)
print("✅ Labels shape:", y.shape)


100%|██████████| 2880/2880 [00:47<00:00, 60.76it/s] 

✅ Features shape: (2880, 40)
✅ Labels shape: (2880,)


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build simple neural network
model = Sequential([
    Dense(256, activation='relu', input_shape=(40,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,521 (170.00 KB)

 Trainable params: 43,521 (170.00 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5123 - loss: 16.5813 - val_accuracy: 0.5139 - val_loss: 1.4829
Epoch 2/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5283 - loss: 3.4031 - val_accuracy: 0.5260 - val_loss: 0.6915
Epoch 3/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5082 - loss: 1.2864 - val_accuracy: 0.5139 - val_loss: 0.6930
Epoch 4/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5157 - loss: 0.8414 - val_accuracy: 0.5139 - val_loss: 0.6926
Epoch 5/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5752 - loss: 0.7165 - val_accuracy: 0.5139 - val_loss: 0.6923
Epoch 6/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5265 - loss: 0.7239 - val_accuracy: 0.5139 - val_loss: 0.6924
Epoch 7/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5443 - loss: 0.7075 - val_accuracy: 0.5139 - val_loss: 0.6922
Epoch 8/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5381 - loss: 0.7028 - val_accuracy: 0.5139 - val_loss

In [13]:
model.save("model.h5")
print("✅ Model saved as model.h5")

✅ Model saved as model.h5


In [14]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6307 - loss: 0.6482 
Test Accuracy: 62.15%


In [15]:
sample = extract_features(files[10]).reshape(1, -1)
prediction = model.predict(sample)
print("🔴 Distress Detected!" if prediction[0][0] > 0.5 else "🟢 Normal Voice")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
🟢 Normal Voice


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# (if you didn't already split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# one-hot encode labels for categorical_crossentropy
y_train_cat = to_categorical(y_train, num_classes=2)
y_test_cat = to_categorical(y_test, num_classes=2)

# Build the model
model = Sequential([
    Dense(256, activation='relu', input_shape=(40,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

# Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
history = model.fit(
    X_train, y_train_cat,
    epochs=40,
    batch_size=32,
    validation_data=(X_test, y_test_cat)
)


Epoch 1/40
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5208 - loss: 13.2051 - val_accuracy: 0.5729 - val_loss: 0.8295
Epoch 2/40
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4962 - loss: 2.6831 - val_accuracy: 0.5278 - val_loss: 0.6902
Epoch 3/40
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5222 - loss: 0.9297 - val_accuracy: 0.5139 - val_loss: 0.6911
Epoch 4/40
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5180 - loss: 0.7847 - val_accuracy: 0.5139 - val_loss: 0.6911
Epoch 5/40
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5443 - loss: 0.7453 - val_accuracy: 0.5139 - val_loss: 0.6914
Epoch 6/40
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5356 - loss: 0.7091 - val_accuracy: 0.5139 - val_loss: 0.6921
Epoch 7/40
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5241 - loss: 0.7233 - val_accuracy: 0.5139 - val_loss: 0.6915
Epoch 8/40
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5408 - loss: 0.7004 - val_accuracy: 0.5139 - val_loss

In [17]:
model.save('distress_model.h5')
print("✅ Model saved successfully!")


✅ Model saved successfully!


In [18]:
import numpy as np
import librosa

# Reload model (optional)
from tensorflow.keras.models import load_model
model = load_model('distress_model.h5')

def predict_audio(file_path):
    y, sr = librosa.load(file_path, duration=3, offset=0.5)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfccs = np.mean(mfcc.T, axis=0)
    x = np.expand_dims(mfccs, axis=0)
    pred = model.predict(x)
    label = np.argmax(pred, axis=1)[0]
    print(f"Prediction: {'🔴 Distress' if label==1 else '🟢 Normal'} | Probabilities:", pred)

# Test on one or two RAVDESS files
sample = files[10]  # pick a random audio file path
print("Testing file:", sample)
predict_audio(sample)


Testing file: /content/ravdess/Actor_05/03-01-08-01-02-01-05.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Prediction: 🟢 Normal | Probabilities: [[0.5350391 0.4649609]]


In [20]:
!ls


distress_model.h5  drive  model.h5  ravdess  sample_data


In [21]:
!ls -lh *.h5


-rw-r--r-- 1 root root 545K Nov  5 09:04 distress_model.h5
-rw-r--r-- 1 root root 543K Nov  5 09:04 model.h5


In [22]:
from google.colab import files

# Download the main trained model
files.download('distress_model.h5')

# (Optional) Download the earlier model version
files.download('model.h5')

# If you also created test_predict.py, download it too
files.download('test_predict.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: Cannot find file: test_predict.py

In [23]:
%%writefile test_predict.py
import sys
import librosa
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('distress_model.h5')

def predict_distress(file_path):
    y, sr = librosa.load(file_path, duration=3, offset=0.5)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfccs = np.mean(mfcc.T, axis=0)
    x = np.expand_dims(mfccs, axis=0)
    pred = model.predict(x)
    label = np.argmax(pred, axis=1)[0]
    return 'distress' if label == 1 else 'normal'

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python test_predict.py <audio_file_path>")
    else:
        file_path = sys.argv[1]
        print("Prediction:", predict_distress(file_path))


Writing test_predict.py


In [24]:
!ls -lh


total 1.1M
-rw-r--r--  1 root root 545K Nov  5 09:04 distress_model.h5
drwx------  5 root root 4.0K Nov  5 09:01 drive
-rw-r--r--  1 root root 543K Nov  5 09:04 model.h5
drwxr-xr-x 27 root root 4.0K Nov  5 09:02 ravdess
drwxr-xr-x  1 root root 4.0K Nov  3 14:39 sample_data
-rw-r--r--  1 root root  710 Nov  5 09:21 test_predict.py


In [25]:
from google.colab import files

# Download your trained models
files.download('distress_model.h5')
files.download('model.h5')

# Download your test script
files.download('test_predict.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>